# Analyse exploratoire

**Analyse et segmentation de clientèle d'un magasin avec campagnes de marketing**
_Jules EXBRAYAT & Abdenour MADANI_

## Import des outils / jeu de données

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from src.config import data_folder
from src.constants import var_categoriques_original as var_categoriques
from src.constants import var_numeriques
from src.utils import init_notebook

In [ ]:
init_notebook()

In [ ]:
df = pd.read_csv(
    f"{data_folder}/marketing_campaign.csv",
    sep="\t",
    index_col="ID",
    parse_dates=True,
)

## Présentation

### Problématique

Dans le rôle d'analystes de données en mission pour un magasin de produits alimentaires, nous chercherons à : 

- Réaliser une segmentation de la clientèle de l'entreprise (clustering),
- Prédire l'efficacité d'une future campagne de marketing (classification binaire)

**Tableau.** Liste de nos objectifs

|  | Mission | Type de tâche            |
| :--- | :--- |:-------------------------|
| Objectif 1 | Segmentation de la clientèle | Clustering non supervisé |
| Objectif 2 | Prédiction d'acceptation à une campagne marketing | Classification binaire   |

**Tableau.** Plan de la présentation

| :--- | :--- |
| **Exploration des données** | Présentation des variables<br>Première visualisation des variables |
| **Pré-traitement des données** | Nettoyage des données<br>Création de variables (Feature Engineering) |
| **Visualisation des données** | Visualisation en fonction de la variable cible |
| **Analyse factorielle** | ACP<br>AFC<br>ACM |
| **ANOVA** | Vérification des hypothèses<br>Test d'ANOVA |
| **Segmentation de clientèle** | Comparaison de différents algorithmes de clusters<br>Visualisation des clusters<br>Description des profils "type" de clients |
| **Prédiction d'acceptation<br>de campagne marketing** | Comparaison de différents modèles de classification<br>Équilibrage des classes<br>Diagnostic du meilleur modèle<br>Conclusion sur l'efficacité de prédiction |

### Jeu de données

In [ ]:
## todo : ajouter une description / une partie pour parler du jeu de données
## "Ce jeu de données contient le profil de plusieurs clients d'une enseigne de grande distribution" \
## "Il contient les données de juillet 2012 à juillet 2014" \
## "Nous disposons des ventes en Vin, etc"

#### Présentation des variables

In [ ]:
df.head()

In [ ]:
print(f"Il y a {df.shape[1]} variables et {df.shape[0]} individus.")

##### Variables

- ID: identifiant du client
- Year_Birth: numérique, année de naissance du client
- Education: qualitative, niveau d'éducation
- Marital_Status: qualitative, statut marital
- Income: numérique, revenu annuel en $
- Kidhome: numérique, nombre d'enfants en bas-âge
- Teenhome: numérique, nombre d'enfants adolescents
- Dt_Customer: date, date à laquelle le client s'est inscrit
- Recency: numérique, nombre de jours depuis le dernier achat
- Complain: catégorique, est-ce que le client s'est plaint les 2 dernières années (0 ou 1)

##### Products

- MntWines: numérique, argent dépensé les 2 dernières années en vin
- MntFruits: numérique, argent dépensé les 2 dernières années en fruits
- MntMeatProducts: numérique, argent dépensé les 2 dernières années en viande
- MntFishProducts: numérique, argent dépensé les 2 dernières années en poisson
- MntSweetProducts: numérique, argent dépensé les 2 dernières années en bonbons / gâteaux
- MntGoldProds: numérique, argent dépensé les 2 dernières années en bijoux / or

##### Promotion

- NumDealsPurchases: numérique, nombre d'achats effectués avec une promotion
- AcceptedCmp1: catégorique, le client a-t-il acheté durant la campagne promotionnelle numéro 1 (1 s'il a acheté, 0 sinon)
- AcceptedCmp2: pareil pour la campagne numéro 2
- AcceptedCmp3: pareil pour la campagne numéro 3
- AcceptedCmp4: pareil pour la campagne numéro 4
- AcceptedCmp5: pareil pour la campagne numéro 5
- Response: catégorique, 1 si le client a acheté durant la dernière campagne, 0 sinon (potentielle variable à prédire)

##### Place

- NumWebPurchases: numérique, nombre d'achats effectués sur le site Internet
- NumCatalogPurchases: numérique, nombre d'achats effectués via le catalogue
- NumStorePurchases: numérique, nombre d'achats effectués en magasin
- NumWebVisitsMonth: numérique, nombre de visites sur le site Internet le dernier mois


Nous séparons les variables numériques des variables catégoriques pour plus de commodités.

Nous convertissons les variables catégoriques en type `category`. (Nous les convertissons au préalable en type `string` car cela facilite l'affichage de la légende avec Matplotlib et Seaborn)

In [ ]:
df[var_categoriques] = df[var_categoriques].astype(str).astype("category")

In [ ]:
## todo: convertir en int la variable Income (qui est float)

Nous convertissons les variables au format date.

In [ ]:
df["Dt_Customer"] = pd.to_datetime(df["Dt_Customer"], format="%d-%m-%Y")

In [ ]:
df.info()

Nous avons 11 variables catégoriques, 16 variables quantitatives (dont 15 entières) ainsi qu'une variable de type date.

## Découverte des données

### Analyse univariée

In [ ]:
df[var_numeriques].describe()

In [ ]:
df[var_categoriques].describe()

In [ ]:
## todo: commenter

### Visualisation

#### Variables numériques

In [ ]:
for var in var_numeriques:
    _, ax = plt.subplots(1, 2, figsize=(8, 2))
    sns.boxplot(df[var], width=0.25, ax=ax[0])
    sns.histplot(df[var], kde=True, ax=ax[1])
    plt.show()

In [ ]:
## todo: commenter les distributions et boxplots

In [ ]:
plt.figure(figsize=(8, 8))
sns.heatmap(
    df[var_numeriques].corr()[df[var_numeriques].corr().abs() > 0.5],
    annot=True,
    cmap="BrBG",
    linewidths=0.5,
    vmax=1,
    vmin=-1,
)

In [ ]:
## todo: commenter la matrice de corrélation

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(15, 4))

ax[0].set_title("Nombre de valeurs présentes")
df.notna().sum()[var_numeriques].plot.barh(ax=ax[0])

ax[1].set_title("Valeurs manquantes")
sns.heatmap(df[var_numeriques].isna(), cbar=False, ax=ax[1])

Nous observons qu'il n'y a quasiment pas de valeurs manquantes parmi les variables numériques.

#### Variables catégoriques

In [ ]:
for var in var_categoriques:
    if df[var].nunique() > 3:
        sns.histplot(y=df[var])
    else:
        plt.figure(figsize=(4, 2))
        sns.histplot(df[var], shrink=0.3)
    plt.show()

In [ ]:
df[
    df[var_categoriques].columns[df[var_categoriques].nunique() > 3]
].value_counts().plot(kind="bar")

In [ ]:
## todo: commenter

##### Valeurs manquantes

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(15, 4))

ax[0].set_title("Nombre de valeurs présentes")
df.notna().sum()[var_categoriques].plot.barh(ax=ax[0])

ax[1].set_title("Valeurs manquantes")
sns.heatmap(df[var_categoriques].isna(), cbar=False, ax=ax[1])

Nous observons l'absence de valeurs manquantes parmi les variables catégoriques.

#### Dates

In [ ]:
## todo

In [ ]:
df["Dt_Customer"].hist(bins=50)

## Sauvegarde du Dataframe

In [ ]:
df.to_csv(f"{data_folder}/data.csv")